In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument  
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import multiprocessing
from sklearn import utils
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from spacy.lang.en import English
from spacy import displacy
import en_core_web_sm
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import string
import numpy as np
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
data = dict()
filenames=list()
cleaned_text=list()
label=list()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/My Drive/BE PROJECT/Lawgical Final/LabelledDataset/'

In [ ]:
# msk = np.random.rand(len(df)) < 0.7
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')


In [ ]:
#list of tagged document that is an input to the doc2vec model
tagged_data = [TaggedDocument(words=word_tokenize(_d['Judgement'].lower()), tags=[str(i)]) for i, _d in train.iterrows()]
tagged_data_test = [TaggedDocument(words=word_tokenize(_d['Judgement'].lower()), tags=[str(i)]) for i, _d in test.iterrows()]


In [ ]:
type(tagged_data)#example how to iterate
print(tagged_data[1])

TaggedDocument(['1', 'petitioner', 'wife', 'detenuealthaf', 's/o', 'detain', 'provision', 'offender', 'gamblers', 'goondas', 'immoral', 'traffic', 'offender', 'slum', 'grabbers', 'act', 'call', 'act', 'sake', 'brevity', '2', 'second', 'respondentcommissioner', 'pass', 'order', 'detention', 'translation', 'say', 'order', 'furnish', 'detenu', 'date', 'vide', 'annexure', 'a1', 'respectively', 'ground', 'detention', 'along', 'translation', 'furnish', 'detenu', 'annexure', 'b', 'b1', 'respectively', 'detention', 'approve', 'order', 'date', 'vide', '3', 'nutshell', 'state', 'ground', 'detention', 'detenu', 'involve', 'several', 'criminal', 'activity', 'within', 'limit', 'damage', 'property', 'obstruct', 'police', 'discharge', 'duty', 'cause', 'injury', 'commit', 'act', 'kidnap', 'extortion', 'dacoity', 'attempt', 'murder', 'collusion', 'antisocial', 'element', 'result', 'open', 'rowdy', 'sheet', 'detenu', '14/05/2010', 'activity', 'detenu', 'create', 'problem', 'law', 'abide', 'citizen', 'th

In [ ]:
max_epochs = 10
vec_size = 1000
alpha = 0.050
#instantiate the model

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

model.build_vocab([x for x in tagged_data])

In [ ]:
#train the model
model.train(utils.shuffle([x for x in tagged_data]), total_examples=len(tagged_data), epochs=max_epochs)


In [ ]:
vector = model.infer_vector(tagged_data_test[6][0])#take an example to show the similar documents in corpus 
sims1 = model.docvecs.most_similar([vector]) #gives you top 10 document tags and their cosine similarity
print(sims1)




[('138', 0.5108263492584229), ('62', 0.507188081741333), ('110', 0.4987693428993225), ('173', 0.49692502617836), ('76', 0.49039992690086365), ('39', 0.4888373017311096), ('131', 0.48684951663017273), ('79', 0.4787421226501465), ('53', 0.4713681638240814), ('116', 0.4692420959472656)]
['1these', 'criminal', 'writ', 'petition', 'challenge', 'order', 'detention', 'date', 'admit', '04/06/', 'time', 'speaking', 'order', 'court', 'direct', 'respondent', '3', 'joint', 'secretary', 'consider', 'writ', 'petition', 'representation', 'detenue', 'article', '225', 'constitution', 'produce', 'court', 'file', 'concern', 'consideration', 'along', 'order', 'pass', 'thereon', 'rule', 'make', 'returnable', 'need', 'go', 'circumstance', 'thereafter', 'matter', 'come', 'place', 'u', 'alternate', 'bench', 'however', 'learn', 'counsel', 'appear', 'respondent', 'no', 'l', '1', '3', 'matter', 'outset', 'urge', 'meeting', 'schedule', 'therefore', 'consideration', 'present', 'matter', 'postpone', 'dispute', 'per

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
#print name and the case index which can be used in the server to fetch the case
for i in sims1:
  print(train.iloc[int(i[0])][1]+"-"+str(train.iloc[int(i[0])][0]))


Mohammed Khwaja Ameer Alias Ameer Alias Faizaan V. State Of Karnataka-146
Prabhakar Rao @ Sanaka Prabhakara Rao V. State Of Karnataka And Another-66
Girish V. The State Of Karnataka By Yeshwanthpura Police Station, Represented By State Public Prosecutor-116
Guruswamy And Others V. State Of Karnataka-183
Arun & Others V. State Of Karnataka, Reptd. By State Public Prosecutor-80
Patil V. The State Of Karnataka-40
M.S. Ramesh V. The State Of Karnataka, Represented By State Public Prosecutor, Bengaluru-138
Pavan V. State-83
V. Vijaya Shetty V. The State By Byndoor Police Station, Represented By Learned Spp, High Court...-56
Puttamadamma V. The State Of Karnataka-123
